Importando base de dados:

In [1]:
import pandas as pd
url = 'https://caelum-online-public.s3.amazonaws.com/2928-transformacao-manipulacao-dados/dados_vendas_clientes.json'
dados_vendas = pd.read_json(url)
dados_vendas.head()

,dados_vendas
0,"{'Data de venda': '06/06/2022', 'Cliente': ['@..."
1,"{'Data de venda': '07/06/2022', 'Cliente': ['I..."
2,"{'Data de venda': '08/06/2022', 'Cliente': ['I..."
3,"{'Data de venda': '09/06/2022', 'Cliente': ['J..."
4,"{'Data de venda': '10/06/2022', 'Cliente': ['M..."


Normalizando dados:

In [2]:
dados_vendas = pd.json_normalize(dados_vendas['dados_vendas'])
dados_vendas.head()

,Data de venda,Cliente,Valor da compra
0,06/06/2022,"[@ANA _LUCIA 321, DieGO ARMANDIU 210, DieGO AR...","[R$ 836,5, R$ 573,33, R$ 392,8, R$ 512,34]"
1,07/06/2022,"[Isabely JOanes 738, Isabely JOanes 738, Isabe...","[R$ 825,31, R$ 168,07, R$ 339,18, R$ 314,69]"
2,08/06/2022,"[Isabely JOanes 738, JOãO Gabriel 671, Julya m...","[R$ 682,05, R$ 386,34, R$ 622,65, R$ 630,79]"
3,09/06/2022,"[Julya meireles 914, MaRIA Julia 444, MaRIA Ju...","[R$ 390,3, R$ 759,16, R$ 334,47, R$ 678,78]"
4,10/06/2022,"[MaRIA Julia 444, PEDRO PASCO 812, Paulo castr...","[R$ 314,24, R$ 311,15, R$ 899,16, R$ 885,24]"


Removendo os dados em listas do DataFrame

In [3]:
dados_vendas = dados_vendas.explode(['Cliente', 'Valor da compra'])
dados_vendas.head()

,Data de venda,Cliente,Valor da compra
0,06/06/2022,@ANA _LUCIA 321,"R$ 836,5"
0,06/06/2022,DieGO ARMANDIU 210,"R$ 573,33"
0,06/06/2022,DieGO ARMANDIU 210,"R$ 392,8"
0,06/06/2022,DieGO ARMANDIU 210,"R$ 512,34"
1,07/06/2022,Isabely JOanes 738,"R$ 825,31"


Corrigindo os valores de rotulo (Index) do DataFrame:

In [4]:
dados_vendas.reset_index(inplace=True,drop=True)

In [5]:
dados_vendas.head()

,Data de venda,Cliente,Valor da compra
0,06/06/2022,@ANA _LUCIA 321,"R$ 836,5"
1,06/06/2022,DieGO ARMANDIU 210,"R$ 573,33"
2,06/06/2022,DieGO ARMANDIU 210,"R$ 392,8"
3,06/06/2022,DieGO ARMANDIU 210,"R$ 512,34"
4,07/06/2022,Isabely JOanes 738,"R$ 825,31"


Verificando os tipos dos dados:

In [6]:
dados_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Data de venda    20 non-null     object
 1   Cliente          20 non-null     object
 2   Valor da compra  20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


Removendo caracteres da coluna "Valor da compra"

In [7]:
dados_vendas['Valor da compra'] = dados_vendas['Valor da compra'].apply(lambda x: x.replace('R$','').replace(',','.').strip())

In [8]:
dados_vendas.head()

,Data de venda,Cliente,Valor da compra
0,06/06/2022,@ANA _LUCIA 321,836.5
1,06/06/2022,DieGO ARMANDIU 210,573.33
2,06/06/2022,DieGO ARMANDIU 210,392.8
3,06/06/2022,DieGO ARMANDIU 210,512.34
4,07/06/2022,Isabely JOanes 738,825.31


Transformando a coluna "Valor da compra" em Float:

In [9]:
import numpy as np
dados_vendas['Valor da compra'] = dados_vendas['Valor da compra'].astype(np.float64)

In [10]:
dados_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data de venda    20 non-null     object 
 1   Cliente          20 non-null     object 
 2   Valor da compra  20 non-null     float64
dtypes: float64(1), object(2)
memory usage: 608.0+ bytes


Fazendo o ajuste da coluna "Cliente":

In [11]:
dados_vendas['Cliente'] = dados_vendas['Cliente'].str.lower()

In [12]:
dados_vendas.head()

,Data de venda,Cliente,Valor da compra
0,06/06/2022,@ana _lucia 321,836.50
1,06/06/2022,diego armandiu 210,573.33
2,06/06/2022,diego armandiu 210,392.80
3,06/06/2022,diego armandiu 210,512.34
4,07/06/2022,isabely joanes 738,825.31


In [17]:
dados_vendas['Cliente'] = dados_vendas['Cliente'].str.replace('[^a-zà-ü ]', '', regex=True)

Transformando a coluna para o formato datetime:

In [18]:
dados_vendas['Data de venda'] = pd.to_datetime(dados_vendas['Data de venda'])

Escolhendo a melhor formatação para essa coluna:

In [23]:
dados_vendas['Data de venda'] = dados_vendas['Data de venda'].dt.strftime('%m-%d-%y')

In [26]:
dados_vendas.head(6)

,Data de venda,Cliente,Valor da compra
0,06-06-22,ana lucia,836.50
1,06-06-22,diego armandiu,573.33
2,06-06-22,diego armandiu,392.80
3,06-06-22,diego armandiu,512.34
4,07-06-22,isabely joanes,825.31
5,07-06-22,isabely joanes,168.07


Descobrindo os clientes que mais gastaram com produtos:

In [34]:
dados_vendas.groupby('Cliente')[['Valor da compra']].sum()

,Valor da compra
Cliente,
ana lucia,836.50
diego armandiu,1478.47
isabely joanes,2329.30
joão gabriel,386.34
julya meireles,1643.74
maria julia,2086.65
paulo castro,899.16
pedro pasco,311.15
thiago fritzz,885.24
